
Automated Face Recognition System Enhanced by AI-driven Group Decision-Making



Import necessary libraries

In [ ]:
!pip install face_recognition
import face_recognition
import cv2
import os
from google.colab.patches import cv2_imshow
import numpy as np
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=a567d76c4f35d9de76ca2ea719b3b3eb783f222fcabf976ca3bd5cc918ebd7ff
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


# Mount Google Drive

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the target face image
#  Encode the target face

In [ ]:

target_image = face_recognition.load_image_file("/content/drive/MyDrive/MastersProject/target/image_1.jpg")

target_face_encoding = face_recognition.face_encodings(target_image)[0]

1. Balanced Confidence Target Biased

In [ ]:
# Function to extract numeric part from file name
def extract_numeric_part(file_name):
    return int(re.search(r'\d+', file_name).group())

# Function to perform face recognition and confidence calculation
def bal_confidence_t_model(target_image_path, crowded_image_path, threshold, results_list):
    target_image = face_recognition.load_image_file(target_image_path)
    target_face_encoding = face_recognition.face_encodings(target_image)

    if len(target_face_encoding) == 0:
        return

    target_face_encoding = target_face_encoding[0]

    # Load crowded image and detect face locations and encodings
    crowded_image = cv2.imread(crowded_image_path)
    face_locations = face_recognition.face_locations(crowded_image)
    face_encodings = face_recognition.face_encodings(crowded_image, face_locations)

    min_distance = float('inf')
    min_distance_face = None

    # Compare each detected face encoding with the target face encoding
    for (face_location, face_encoding) in zip(face_locations, face_encodings):
        # Calculate the Euclidean distance between face encodings
        distance = np.linalg.norm(target_face_encoding - face_encoding)

        if distance < min_distance:
            min_distance = distance
            min_distance_face = (face_location, distance)

    if min_distance_face is not None:
        (top, right, bottom, left), distance = min_distance_face
        if distance < threshold:

            # Calculate confidence for target face
            confidence = 2 / (1 + np.exp(-10 * (threshold - distance) / threshold)) - 1
            result = f"Target {confidence:.1f}"
        else:
            # Calculate confidence for non-target face
            confidence = 2 / (1 + np.exp(-10 * (distance - threshold) / (1 - threshold))) - 1
            result = f"NonTarget {confidence:.1f}"

        # Draw bounding box and label on the image
        cv2.rectangle(crowded_image, (left, top), (right, bottom), (0, 255, 0), 2)
        label = f"{result}"
        cv2.putText(crowded_image, label, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    else:
        result = "NonTarget 0.0"
        # If no face is detected, show result as NonTarget with confidence 0.0
        cv2.putText(crowded_image, "Result: NonTarget 0.0", (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Display the image with bounding box and label
    cv2_imshow(crowded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Append result to the results list
    results_list.append(result)

# Path to the target image
target_image_path = "/content/drive/MyDrive/MastersProject/target/image_1.jpg"
# Directory containing crowded images
crowded_images_directory = "/content/drive/MyDrive/MastersProject/Imagessorted/"
# Threshold for confidence calculation
threshold = 0.6

# Get list of files in the crowded images directory
crowded_image_files = os.listdir(crowded_images_directory)

# Extract numeric part from each file name for sorting
numeric_file_names = [int(file_name.split('_')[-1].split('.')[0]) for file_name in crowded_image_files]

# Sort files based on the numeric part of file names
sorted_files = [file_name for _, file_name in sorted(zip(numeric_file_names, crowded_image_files))]


# List to store results for all images
all_results_bc_t = []

# Loop through sorted image files
for image_file in sorted_files:
    if image_file.endswith(".jpg") or image_file.endswith(".png") or image_file.endswith(".jpeg"):

        # Construct path to the current image
        crowded_image_path = os.path.join(crowded_images_directory, image_file)

        # List to store results for the current image
        results = []

        # Perform face recognition and confidence calculation
        bal_confidence_t_model(target_image_path, crowded_image_path, threshold, results)

        # Append results for the current image to the list of all results
        all_results_bc_t.append(results)

# Print all results
print(all_results_bc_t)


2. Balanced Confidence NONTARGET BIASED

In [ ]:
# Function to extract numeric part from a file name
def extract_numeric_part(file_name):
    return int(re.search(r'\d+', file_name).group())

# Function to perform face recognition and confidence calculation for non-target faces
def bal_confidence_nt_model(target_image_path, crowded_image_path, threshold, results_list):
    # Load target image and extract its encoding
    target_image = face_recognition.load_image_file(target_image_path)
    target_face_encoding = face_recognition.face_encodings(target_image)

    # Check if no face is detected in the target image
    if len(target_face_encoding) == 0:
        return

    target_face_encoding = target_face_encoding[0]

    # Load crowded image and detect face locations and encodings
    crowded_image = cv2.imread(crowded_image_path)
    face_locations = face_recognition.face_locations(crowded_image)
    face_encodings = face_recognition.face_encodings(crowded_image, face_locations)

    min_distance = float('inf')
    min_distance_face = None

    # Compare each detected face encoding with the target face encoding
    for (face_location, face_encoding) in zip(face_locations, face_encodings):
        # Calculate the Euclidean distance between face encodings
        distance = np.linalg.norm(target_face_encoding - face_encoding)

        if distance < min_distance:
            min_distance = distance
            min_distance_face = (face_location, distance)

    # If a face is detected in the crowded image
    if min_distance_face is not None:
        (top, right, bottom, left), distance = min_distance_face
        # Check if the distance is below the threshold
        if distance < threshold:
            # Calculate confidence for non-target face
            confidence = 2 / (1 + np.exp(-10 * (threshold - distance) / threshold)) - 1
            result = f"Target {confidence:.1f}"
        else:
            # Calculate confidence for non-target face
            confidence = 2 / (1 + np.exp(-10 * (distance - threshold) / (1 - threshold))) - 1
            result = f"NonTarget {confidence:.1f}"
        # Draw bounding box and label on the image
        cv2.rectangle(crowded_image, (left, top), (right, bottom), (0, 255, 0), 2)
        label = f"{result}"
        cv2.putText(crowded_image, label, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    else:
        # If no face is detected, show result as NonTarget with confidence 0.0
        result = "NonTarget 0.0"
        cv2.putText(crowded_image, "Result: NonTarget 0.0", (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Display the image with bounding box and label
    cv2_imshow(crowded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Append result to the results list
    results_list.append(result)


# Path to the target image
target_image_path = "/content/drive/MyDrive/MastersProject/target/image_1.jpg"
# Directory containing crowded images
crowded_images_directory = "/content/drive/MyDrive/MastersProject/Imagessorted/"
# Threshold for confidence calculation
threshold = 0.4

# Get list of files in the crowded images directory
crowded_image_files = os.listdir(crowded_images_directory)
# Extract numeric part from each file name for sorting
numeric_file_names = [int(file_name.split('_')[-1].split('.')[0]) for file_name in crowded_image_files]
# Sort files based on the numeric part of file names
sorted_files = [file_name for _, file_name in sorted(zip(numeric_file_names, crowded_image_files))]

# List to store results for all images
all_results_bc_nt = []

# Loop through sorted image files
for image_file in sorted_files:
    if image_file.endswith(".jpg") or image_file.endswith(".png") or image_file.endswith(".jpeg"):
        # Construct path to the current image
        crowded_image_path = os.path.join(crowded_images_directory, image_file)
        # List to store results for the current image
        results = []
        # Perform face recognition and confidence calculation for non-target faces
        bal_confidence_nt_model(target_image_path, crowded_image_path, threshold, results)
        # Append results for the current image to the list of all results
        all_results_bc_nt.append(results)

# Print all results
print(all_results_bc_nt)


3. High Confidence Unbiased

In [ ]:
# Function to extract numeric part from a file name
def extract_numeric_part(file_name):
    return int(re.search(r'\d+', file_name).group())

# Function to perform face recognition and confidence calculation for high confidence model
def high_confidence_model(target_image_path, crowded_image_path, threshold, results_list):
    # Load target image and extract its encoding
    target_image = face_recognition.load_image_file(target_image_path)
    target_face_encoding = face_recognition.face_encodings(target_image)

    # Check if no face is detected in the target image
    if len(target_face_encoding) == 0:
        return

    target_face_encoding = target_face_encoding[0]

    # Load crowded image and detect face locations and encodings
    crowded_image = cv2.imread(crowded_image_path)
    face_locations = face_recognition.face_locations(crowded_image)
    face_encodings = face_recognition.face_encodings(crowded_image, face_locations)

    min_distance = float('inf')
    min_distance_face = None

    # Compare each detected face encoding with the target face encoding
    for (face_location, face_encoding) in zip(face_locations, face_encodings):
        # Calculate the Euclidean distance between face encodings
        distance = np.linalg.norm(target_face_encoding - face_encoding)

        if distance < min_distance:
            min_distance = distance
            min_distance_face = (face_location, distance)

    # If a face is detected in the crowded image
    if min_distance_face is not None:
        (top, right, bottom, left), distance = min_distance_face
        # Check if the distance is below the threshold
        if distance < threshold:
            # Calculate confidence for target face
            confidence = 2 / (1 + np.exp(-10 * (threshold - distance) / threshold * 1.2)) - 1
            result = f"Target {confidence:.1f}"
        else:
            # Calculate confidence for non-target face
            confidence = 2 / (1 + np.exp(-10 * (distance - threshold) / (1 - threshold) * 1.2)) - 1
            result = f"NonTarget {confidence:.1f}"
        # Draw bounding box and label on the image
        cv2.rectangle(crowded_image, (left, top), (right, bottom), (0, 255, 0), 2)
        label = f"{result}"
        cv2.putText(crowded_image, label, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    else:
        # If no face is detected, show result as NonTarget with confidence 0.0
        result = "NonTarget 0.0"
        cv2.putText(crowded_image, "Result: NonTarget 0.0", (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Display the image with bounding box and label
    cv2_imshow(crowded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Append result to the results list
    results_list.append(result)


# Path to the target image
target_image_path = "/content/drive/MyDrive/MastersProject/target/image_1.jpg"
# Directory containing crowded images
crowded_images_directory = "/content/drive/MyDrive/MastersProject/Imagessorted/"
# Threshold for confidence calculation
threshold = 0.5

# Get list of files in the crowded images directory
crowded_image_files = os.listdir(crowded_images_directory)
# Extract numeric part from each file name for sorting
numeric_file_names = [int(file_name.split('_')[-1].split('.')[0]) for file_name in crowded_image_files]
# Sort files based on the numeric part of file names
sorted_files = [file_name for _, file_name in sorted(zip(numeric_file_names, crowded_image_files))]

# List to store results for all images
all_results_hc = []

# Loop through sorted image files
for image_file in sorted_files:
    if image_file.endswith(".jpg") or image_file.endswith(".png") or image_file.endswith(".jpeg"):
        # Construct path to the current image
        crowded_image_path = os.path.join(crowded_images_directory, image_file)
        # List to store results for the current image
        results = []
        # Perform face recognition and confidence calculation for high confidence model
        high_confidence_model(target_image_path, crowded_image_path, threshold, results)
        # Append results for the current image to the list of all results
        all_results_hc.append(results)

# Print all results
print(all_results_hc)


4. Low Confidence Unbiased

In [ ]:
# Function to extract numeric part from a file name
def extract_numeric_part(file_name):
    return int(re.search(r'\d+', file_name).group())

# Function to perform face recognition and confidence calculation for low confidence model
def low_confidence_model(target_image_path, crowded_image_path, threshold, results_list):
    # Load target image and extract its encoding
    target_image = face_recognition.load_image_file(target_image_path)
    target_face_encoding = face_recognition.face_encodings(target_image)

    # Check if no face is detected in the target image
    if len(target_face_encoding) == 0:
        return

    target_face_encoding = target_face_encoding[0]

    # Load crowded image and detect face locations and encodings
    crowded_image = cv2.imread(crowded_image_path)
    face_locations = face_recognition.face_locations(crowded_image)
    face_encodings = face_recognition.face_encodings(crowded_image, face_locations)

    min_distance = float('inf')
    min_distance_face = None

    # Compare each detected face encoding with the target face encoding
    for (face_location, face_encoding) in zip(face_locations, face_encodings):
        # Calculate the Euclidean distance between face encodings
        distance = np.linalg.norm(target_face_encoding - face_encoding)

        if distance < min_distance:
            min_distance = distance
            min_distance_face = (face_location, distance)

    # If a face is detected in the crowded image
    if min_distance_face is not None:
        (top, right, bottom, left), distance = min_distance_face
        # Check if the distance is below the threshold
        if distance < threshold:
            # Calculate confidence for target face
            confidence = 2 / (1 + np.exp(-10 * (threshold - distance) / threshold * 0.6)) - 1
            result = f"Target {confidence:.1f}"
        else:
            # Calculate confidence for non-target face
            confidence = 2 / (1 + np.exp(-10 * (distance - threshold) / (1 - threshold) * 0.6)) - 1
            result = f"NonTarget {confidence:.1f}"
        # Draw bounding box and label on the image
        cv2.rectangle(crowded_image, (left, top), (right, bottom), (0, 255, 0), 2)
        label = f"{result}"
        cv2.putText(crowded_image, label, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    else:
        # If no face is detected, show result as NonTarget with confidence 0.0
        result = "NonTarget 0.0"
        cv2.putText(crowded_image, "Result: NonTarget 0.0", (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Display the image with bounding box and label
    cv2_imshow(crowded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Append result to the results list
    results_list.append(result)


# Path to the target image
target_image_path = "/content/drive/MyDrive/MastersProject/target/image_1.jpg"
# Directory containing crowded images
crowded_images_directory = "/content/drive/MyDrive/MastersProject/Imagessorted/"
# Threshold for confidence calculation
threshold = 0.5

# Get list of files in the crowded images directory
crowded_image_files = os.listdir(crowded_images_directory)
# Extract numeric part from each file name for sorting
numeric_file_names = [int(file_name.split('_')[-1].split('.')[0]) for file_name in crowded_image_files]
# Sort files based on the numeric part of file names
sorted_files = [file_name for _, file_name in sorted(zip(numeric_file_names, crowded_image_files))]

# List to store results for all images
all_results_lc = []

# Loop through sorted image files
for image_file in sorted_files:
    if image_file.endswith(".jpg") or image_file.endswith(".png") or image_file.endswith(".jpeg"):
        # Construct path to the current image
        crowded_image_path = os.path.join(crowded_images_directory, image_file)
        # List to store results for the current image
        results = []
        # Perform face recognition and confidence calculation for low confidence model
        low_confidence_model(target_image_path, crowded_image_path, threshold, results)
        # Append results for the current image to the list of all results
        all_results_lc.append(results)

# Print all results
print(all_results_lc)


5. High Confidence Target Biased

In [ ]:
# Function to extract numeric part from a file name
def extract_numeric_part(file_name):
    return int(re.search(r'\d+', file_name).group())

# Function to perform face recognition and confidence calculation for high confidence with threshold model
def high_confidence_t_model(target_image_path, crowded_image_path, threshold, results_list):
    # Load target image and extract its encoding
    target_image = face_recognition.load_image_file(target_image_path)
    target_face_encoding = face_recognition.face_encodings(target_image)

    # Check if no face is detected in the target image
    if len(target_face_encoding) == 0:
        return

    target_face_encoding = target_face_encoding[0]

    # Load crowded image and detect face locations and encodings
    crowded_image = cv2.imread(crowded_image_path)
    face_locations = face_recognition.face_locations(crowded_image)
    face_encodings = face_recognition.face_encodings(crowded_image, face_locations)

    min_distance = float('inf')
    min_distance_face = None

    # Compare each detected face encoding with the target face encoding
    for (face_location, face_encoding) in zip(face_locations, face_encodings):
        # Calculate the Euclidean distance between face encodings
        distance = np.linalg.norm(target_face_encoding - face_encoding)

        if distance < min_distance:
            min_distance = distance
            min_distance_face = (face_location, distance)

    # If a face is detected in the crowded image
    if min_distance_face is not None:
        (top, right, bottom, left), distance = min_distance_face
        # Check if the distance is below the threshold
        if distance < threshold:
            # Calculate confidence for target face
            confidence = 2 / (1 + np.exp(-10 * (threshold - distance) / threshold * 1.2)) - 1
            result = f"Target {confidence:.1f}"
        else:
            # Calculate confidence for non-target face
            confidence = 2 / (1 + np.exp(-10 * (distance - threshold) / (1 - threshold) * 1.2)) - 1
            result = f"NonTarget {confidence:.1f}"
        # Draw bounding box and label on the image
        cv2.rectangle(crowded_image, (left, top), (right, bottom), (0, 255, 0), 2)
        label = f"{result}"
        cv2.putText(crowded_image, label, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    else:
        # If no face is detected, show result as NonTarget with confidence 0.0
        result = "NonTarget 0.0"
        cv2.putText(crowded_image, "Result: NonTarget 0.0", (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Display the image with bounding box and label
    cv2_imshow(crowded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Append result to the results list
    results_list.append(result)


# Path to the target image
target_image_path = "/content/drive/MyDrive/MastersProject/target/image_1.jpg"
# Directory containing crowded images
crowded_images_directory = "/content/drive/MyDrive/MastersProject/Imagessorted/"
# Threshold for confidence calculation
threshold = 0.6

# Get list of files in the crowded images directory
crowded_image_files = os.listdir(crowded_images_directory)
# Extract numeric part from each file name for sorting
numeric_file_names = [int(file_name.split('_')[-1].split('.')[0]) for file_name in crowded_image_files]
# Sort files based on the numeric part of file names
sorted_files = [file_name for _, file_name in sorted(zip(numeric_file_names, crowded_image_files))]

# List to store results for all images
all_results_hc_t = []

# Loop through sorted image files
for image_file in sorted_files:
    if image_file.endswith(".jpg") or image_file.endswith(".png") or image_file.endswith(".jpeg"):
        # Construct path to the current image
        crowded_image_path = os.path.join(crowded_images_directory, image_file)
        # List to store results for the current image
        results = []
        # Perform face recognition and confidence calculation for high confidence with threshold model
        high_confidence_t_model(target_image_path, crowded_image_path, threshold, results)
        # Append results for the current image to the list of all results
        all_results_hc_t.append(results)

# Print all results
print(all_results_hc_t)


6. High Confidence NonTarget Biased

In [ ]:
# Function to extract numeric part from a file name
def extract_numeric_part(file_name):
    return int(re.search(r'\d+', file_name).group())

# Function to perform face recognition and confidence calculation for high confidence non-target model
def high_confidence_nt_model(target_image_path, crowded_image_path, threshold, results_list):
    # Load target image and extract its encoding
    target_image = face_recognition.load_image_file(target_image_path)
    target_face_encoding = face_recognition.face_encodings(target_image)

    # Check if no face is detected in the target image
    if len(target_face_encoding) == 0:
        return

    target_face_encoding = target_face_encoding[0]

    # Load crowded image and detect face locations and encodings
    crowded_image = cv2.imread(crowded_image_path)
    face_locations = face_recognition.face_locations(crowded_image)
    face_encodings = face_recognition.face_encodings(crowded_image, face_locations)

    min_distance = float('inf')
    min_distance_face = None

    # Compare each detected face encoding with the target face encoding
    for (face_location, face_encoding) in zip(face_locations, face_encodings):
        # Calculate the Euclidean distance between face encodings
        distance = np.linalg.norm(target_face_encoding - face_encoding)

        if distance < min_distance:
            min_distance = distance
            min_distance_face = (face_location, distance)

    # If a face is detected in the crowded image
    if min_distance_face is not None:
        (top, right, bottom, left), distance = min_distance_face
        # Check if the distance is below the threshold
        if distance < threshold:
            # Calculate confidence for target face
            confidence = 2 / (1 + np.exp(-10 * (threshold - distance) / threshold * 1.2)) - 1
            result = f"Target {confidence:.1f}"
        else:
            # Calculate confidence for non-target face
            confidence = 2 / (1 + np.exp(-10 * (distance - threshold) / (1 - threshold) * 1.2)) - 1
            result = f"NonTarget {confidence:.1f}"
        # Draw bounding box and label on the image
        cv2.rectangle(crowded_image, (left, top), (right, bottom), (0, 255, 0), 2)
        label = f"{result}"
        cv2.putText(crowded_image, label, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    else:
        # If no face is detected, show result as NonTarget with confidence 0.0
        result = "NonTarget 0.0"
        cv2.putText(crowded_image, "Result: NonTarget 0.0", (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Display the image with bounding box and label
    cv2_imshow(crowded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Append result to the results list
    results_list.append(result)


# Path to the target image
target_image_path = "/content/drive/MyDrive/MastersProject/target/image_1.jpg"
# Directory containing crowded images
crowded_images_directory = "/content/drive/MyDrive/MastersProject/Imagessorted/"
# Threshold for confidence calculation
threshold = 0.4

# Get list of files in the crowded images directory
crowded_image_files = os.listdir(crowded_images_directory)
# Extract numeric part from each file name for sorting
numeric_file_names = [int(file_name.split('_')[-1].split('.')[0]) for file_name in crowded_image_files]
# Sort files based on the numeric part of file names
sorted_files = [file_name for _, file_name in sorted(zip(numeric_file_names, crowded_image_files))]

# List to store results for all images
all_results_hc_nt = []

# Loop through sorted image files
for image_file in sorted_files:
    if image_file.endswith(".jpg") or image_file.endswith(".png") or image_file.endswith(".jpeg"):
        # Construct path to the current image
        crowded_image_path = os.path.join(crowded_images_directory, image_file)
        # List to store results for the current image
        results = []
        # Perform face recognition and confidence calculation for high confidence non-target model
        high_confidence_nt_model(target_image_path, crowded_image_path, threshold, results)
        # Append results for the current image to the list of all results
        all_results_hc_nt.append(results)

# Print all results
print(all_results_hc_nt)


7. Low Confidence Target Biased

In [ ]:
# Function to extract numeric part from a file name
def extract_numeric_part(file_name):
    return int(re.search(r'\d+', file_name).group())

# Function to perform face recognition and confidence calculation for low confidence target model
def low_confidence_t_model(target_image_path, crowded_image_path, threshold, results_list):
    # Load target image and extract its encoding
    target_image = face_recognition.load_image_file(target_image_path)
    target_face_encoding = face_recognition.face_encodings(target_image)

    # Check if no face is detected in the target image
    if len(target_face_encoding) == 0:
        return

    target_face_encoding = target_face_encoding[0]

    # Load crowded image and detect face locations and encodings
    crowded_image = cv2.imread(crowded_image_path)
    face_locations = face_recognition.face_locations(crowded_image)
    face_encodings = face_recognition.face_encodings(crowded_image, face_locations)

    min_distance = float('inf')
    min_distance_face = None

    # Compare each detected face encoding with the target face encoding
    for (face_location, face_encoding) in zip(face_locations, face_encodings):
        # Calculate the Euclidean distance between face encodings
        distance = np.linalg.norm(target_face_encoding - face_encoding)

        if distance < min_distance:
            min_distance = distance
            min_distance_face = (face_location, distance)

    # If a face is detected in the crowded image
    if min_distance_face is not None:
        (top, right, bottom, left), distance = min_distance_face
        # Check if the distance is below the threshold
        if distance < threshold:
            # Calculate confidence for target face
            confidence = 2 / (1 + np.exp(-10 * (threshold - distance) / threshold * 0.6)) - 1
            result = f"Target {confidence:.1f}"
        else:
            # Calculate confidence for non-target face
            confidence = 2 / (1 + np.exp(-10 * (distance - threshold) / (1 - threshold) * 0.6)) - 1
            result = f"NonTarget {confidence:.1f}"
        # Draw bounding box and label on the image
        cv2.rectangle(crowded_image, (left, top), (right, bottom), (0, 255, 0), 2)
        label = f"{result}"
        cv2.putText(crowded_image, label, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    else:
        # If no face is detected, show result as NonTarget with confidence 0.0
        result = "NonTarget 0.0"
        cv2.putText(crowded_image, "Result: NonTarget 0.0", (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Display the image with bounding box and label
    cv2_imshow(crowded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Append result to the results list
    results_list.append(result)

# Path to the target image
target_image_path = "/content/drive/MyDrive/MastersProject/target/image_1.jpg"
# Directory containing crowded images
crowded_images_directory = "/content/drive/MyDrive/MastersProject/Imagessorted/"
# Threshold for confidence calculation
threshold = 0.6

# Get list of files in the crowded images directory
crowded_image_files = os.listdir(crowded_images_directory)
# Extract numeric part from each file name for sorting
numeric_file_names = [int(file_name.split('_')[-1].split('.')[0]) for file_name in crowded_image_files]
# Sort files based on the numeric part of file names
sorted_files = [file_name for _, file_name in sorted(zip(numeric_file_names, crowded_image_files))]

# List to store results for all images
all_results_lc_t = []

# Loop through sorted image files
for image_file in sorted_files:
    if image_file.endswith(".jpg") or image_file.endswith(".png") or image_file.endswith(".jpeg"):
        # Construct path to the current image
        crowded_image_path = os.path.join(crowded_images_directory, image_file)
        # List to store results for the current image
        results = []
        # Perform face recognition and confidence calculation for low confidence target model
        low_confidence_t_model(target_image_path, crowded_image_path, threshold, results)
        # Append results for the current image to the list of all results
        all_results_lc_t.append(results)

# Print all results
print(all_results_lc_t)


8. Low Confidence Nontarget Biased

In [ ]:
# Function to extract numeric part from a file name
def extract_numeric_part(file_name):
    return int(re.search(r'\d+', file_name).group())

# Function to perform face recognition and confidence calculation for low confidence non-target model
def low_confidence_nt_model(target_image_path, crowded_image_path, threshold, results_list):
    # Load target image and extract its encoding
    target_image = face_recognition.load_image_file(target_image_path)
    target_face_encoding = face_recognition.face_encodings(target_image)

    # Check if no face is detected in the target image
    if len(target_face_encoding) == 0:
        return

    target_face_encoding = target_face_encoding[0]

    # Load crowded image and detect face locations and encodings
    crowded_image = cv2.imread(crowded_image_path)
    face_locations = face_recognition.face_locations(crowded_image)
    face_encodings = face_recognition.face_encodings(crowded_image, face_locations)

    min_distance = float('inf')
    min_distance_face = None

    # Compare each detected face encoding with the target face encoding
    for (face_location, face_encoding) in zip(face_locations, face_encodings):
        # Calculate the Euclidean distance between face encodings
        distance = np.linalg.norm(target_face_encoding - face_encoding)

        if distance < min_distance:
            min_distance = distance
            min_distance_face = (face_location, distance)

    # If a face is detected in the crowded image
    if min_distance_face is not None:
        (top, right, bottom, left), distance = min_distance_face
        # Check if the distance is below the threshold
        if distance < threshold:
            # Calculate confidence for target face
            confidence = 2 / (1 + np.exp(-10 * (threshold - distance) / threshold * 0.6)) - 1
            result = f"Target {confidence:.1f}"
        else:
            # Calculate confidence for non-target face
            confidence = 2 / (1 + np.exp(-10 * (distance - threshold) / (1 - threshold) * 0.6)) - 1
            result = f"NonTarget {confidence:.1f}"
        # Draw bounding box and label on the image
        cv2.rectangle(crowded_image, (left, top), (right, bottom), (0, 255, 0), 2)
        label = f"{result}"
        cv2.putText(crowded_image, label, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    else:
        # If no face is detected, show result as NonTarget with confidence 0.0
        result = "NonTarget 0.0"
        cv2.putText(crowded_image, "Result: NonTarget 0.0", (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Display the image with bounding box and label
    cv2_imshow(crowded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Append result to the results list
    results_list.append(result)

# Path to the target image
target_image_path = "/content/drive/MyDrive/MastersProject/target/image_1.jpg"
# Directory containing crowded images
crowded_images_directory = "/content/drive/MyDrive/MastersProject/Imagessorted/"
# Threshold for confidence calculation
threshold = 0.4

# Get list of files in the crowded images directory
crowded_image_files = os.listdir(crowded_images_directory)
# Extract numeric part from each file name for sorting
numeric_file_names = [int(file_name.split('_')[-1].split('.')[0]) for file_name in crowded_image_files]
# Sort files based on the numeric part of file names
sorted_files = [file_name for _, file_name in sorted(zip(numeric_file_names, crowded_image_files))]

# List to store results for all images
all_results_lc_nt = []

# Loop through sorted image files
for image_file in sorted_files:
    if image_file.endswith(".jpg") or image_file.endswith(".png") or image_file.endswith(".jpeg"):
        # Construct path to the current image
        crowded_image_path = os.path.join(crowded_images_directory, image_file)
        # List to store results for the current image
        results = []
        # Perform face recognition and confidence calculation for low confidence non-target model
        low_confidence_nt_model(target_image_path, crowded_image_path, threshold, results)
        # Append results for the current image to the list of all results
        all_results_lc_nt.append(results)

# Print all results
print(all_results_lc_nt)
